In [1]:
# Install necessary libraries
!pip install transformers torch matplotlib seaborn tqdm scikit-learn pandas nltk bitsandbytes accelerate sentence-transformers rouge-score

# Import libraries
import json
import time
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
from difflib import SequenceMatcher
from accelerate import init_empty_weights, infer_auto_device_map
from transformers import BitsAndBytesConfig

# Download NLTK data for tokenization
nltk.download('punkt')

# Load the dataset from Kaggle's input directory
with open('/kaggle/input/statss/stats_problems_dataset.json', 'r') as f:  # Corrected file path
    dataset = json.load(f)

# Convert dataset to DataFrame for stratified sampling (300-400 problems)
df = pd.DataFrame(dataset)
sampled_dataset = df.groupby('difficulty', group_keys=False).apply(
    lambda x: x.sample(min(len(x), 400 // df['difficulty'].nunique()), random_state=42)
).to_dict('records')

# Load the SentenceTransformer model for embedding similarity
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Initialize ROUGE scorer
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to calculate traditional similarity (BLEU, ROUGE) and embedding similarity
def calculate_similarity(text1, text2):
    return SequenceMatcher(None, text1.lower(), text2.lower()).ratio()

# Function to calculate BLEU score with smoothing
def calculate_bleu_score(reference, hypothesis):
    smoothie = SmoothingFunction().method4
    return sentence_bleu([reference.split()], hypothesis.split(), smoothing_function=smoothie)

def calculate_rouge(reference, hypothesis):
    scores = rouge_scorer.score(reference, hypothesis)
    return scores

def evaluate_answer(generated, correct):
    # Calculate similarity metrics
    similarity = calculate_similarity(generated, correct)
    bleu_score = calculate_bleu_score(correct, generated)
    rouge_score = calculate_rouge(correct, generated)

    # Embedding-based similarity
    generated_embedding = embedding_model.encode([generated])
    correct_embedding = embedding_model.encode([correct])
    embedding_similarity = cosine_similarity(generated_embedding, correct_embedding)[0][0]

    # Evaluation threshold
    is_correct = embedding_similarity > 0.4  # Threshold can be tuned
    return {
        "is_correct": is_correct,
        "similarity": similarity,
        "embedding_similarity": embedding_similarity,
        "bleu_score": bleu_score,
        "rouge_1": rouge_score['rouge1'].fmeasure,
        "rouge_2": rouge_score['rouge2'].fmeasure,
        "rouge_l": rouge_score['rougeL'].fmeasure
    }

def evaluate_model(model, tokenizer, dataset):
    results = []
    for problem in tqdm(dataset):
        prompt = f"Solve the following statistics problem step-by-step, including all necessary calculations and detailed explanations:\n\nProblem: {problem['problem']}\n\nSolution:"
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)
        
        try:
            start_time = time.time()
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=350, num_return_sequences=1)
            generated_solution = tokenizer.decode(outputs[0], skip_special_tokens=True)
            end_time = time.time()

            evaluation_results = evaluate_answer(generated_solution, problem['solution']['conclusion'])

            results.append({
                "model_name": "google/flan-t5-xl",
                "topic": problem["topic"],
                "difficulty": problem["difficulty"],
                "generated_solution": generated_solution,
                "correct_solution": problem["solution"]["conclusion"],
                "time_taken": end_time - start_time,
                **evaluation_results
            })
        except Exception as e:
            print(f"Error processing problem: {e}")
    
    return results

# Step 1: Load the Flan-T5-XL model
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

# Load the Flan-T5-XL model and tokenizer
model_name = "google/flan-t5-xl"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.eval()

# Step 2: Evaluate model on the sampled dataset
results = evaluate_model(model, tokenizer, sampled_dataset)

# Step 3: Normalize numpy types
def normalize_types(results):
    for result in results:
        for key, value in result.items():
            if isinstance(value, np.generic):
                result[key] = value.item()
    return results

# Step 4: Normalize and save results
normalized_results = normalize_types(results)

with open('/kaggle/working/flan_t5_xl_results.json', 'w') as f:
    json.dump(normalized_results, f)

print("Evaluation complete. Results saved to flan_t5_xl_results.json")
torch.cuda.empty_cache()

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 11.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 16.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=ac570b87acf912798811a2cb937ec0bb5bf99d81b28fd13bed27c5a955212d5c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/tmp/ipykernel_36/3705606973.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_dataset = df.groupby('difficulty', group_keys=False).apply(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

100%|█████████▉| 319/320 [1:29:21<00:23, 23.59s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 320/320 [1:29:23<00:00, 16.76s/it]

Evaluation complete. Results saved to flan_t5_xl_results.json
